In [ ]:
#import dependencies
import pandas as pd
import numpy as np
import os
import gmaps
import json
import requests
from pprint import pprint
import us

#import API key
from config_template import gkey, weather_api_key
gmaps.configure(api_key=gkey)

#output file path for data resource files and figures
outputFilePath = os.path.join("Output-Data")

In [ ]:
#mannually imported information from text file online to csv
#mannually did some cleaning with the data in excel
#we do not know web scraping yet so was unable to utilize that for infomration

In [ ]:
#read the sunshine csv file into a dataframe
sunshine_df = pd.read_csv(os.path.join("..","potential_data_sets","NOAA_Sunshine_AveragePercentPossible.csv"))
sunshine_df

In [ ]:
#clean up the column names
sunshine_df = sunshine_df.rename(columns={
    "CITY":"City",
    "STATE":"State",
    "ANNUAL % AVG POSSIBLE SUNSHINE":"Annual % Average Possible Sunshine",
    "NO. OF YEARS OF DATA":"# of Data Years"
})

In [ ]:
#check the datatypes
sunshine_df.dtypes

In [ ]:
#strip the % column of the % symbol and convert to float
sunshine_df["Annual % Average Possible Sunshine"] = sunshine_df["Annual % Average Possible Sunshine"].map(lambda x: x.rstrip('%'))
sunshine_df["Annual % Average Possible Sunshine"] = sunshine_df["Annual % Average Possible Sunshine"].astype(float)
sunshine_df

In [ ]:
#credit to: https://code.activestate.com/recipes/577305-python-dictionary-of-us-states-and-territories/
#added in some territories mannually to this dictionary
#dictionary of state abbreviations:
states_dict = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'FM': 'Federal States of Micronesia',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MH': 'Marshall Islands',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'PW': 'Palau',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [ ]:
state=[]
for row in range(len(sunshine_df)):
    try:
        state.append(states_dict[sunshine_df['State'][row]])
    except:
        state.append(sunshine_df['State'][row])

cleanSunshine_df = sunshine_df.copy()
cleanSunshine_df["State"]=state
cleanSunshine_df

In [ ]:
#clean the casing of cities
cities = cleanSunshine_df['City']
cities = cities.str.title()
cleanSunshine_df['City']=cities
cleanSunshine_df

In [ ]:
#google maps geocoding url
url_geocoding = "https://maps.googleapis.com/maps/api/geocode/json?"

#find the latitude and longitude of the cities
lat = []
lng = []

#define countrycode
currentCountry = "US"

#loop through the cities_df to retrieve the weather information
for index, row in cleanSunshine_df.iterrows():
    currentCity = row["City"]
    currentState = row["State"]
    target_url = f"{url_geocoding}address={currentCity},{currentState}&region={currentCountry}&key={gkey}"
    response = requests.get(target_url).json()
    
    try:
        lat.append(response['results'][0]['geometry']['location']['lat'])
        lng.append(response['results'][0]['geometry']['location']['lng'])
        print(f"Processeing Record | {currentCity}")
    except:
        lat.append(np.nan)
        lng.append(np.nan)
        print(f"City not found. Skipping {currentCity}...")

#update latitude and longitude to actually match the city
cleanSunshine_df["Latitude"] = lat
cleanSunshine_df["Longitude"] = lng

In [ ]:
#export results to csv file
cleanSunshine_df.to_csv(os.path.join(outputFilePath,"cleanSunshine.csv"),index=False)

cleanSunshine_df

In [ ]:
#clean the data set of null values
sunshineLocations_df = cleanSunshine_df.dropna().reset_index(drop=True)
sunshineLocations_df

In [ ]:
#collect the weightsof %
sunshine = sunshineLocations_df["Annual % Average Possible Sunshine"].astype(float)

#max of % sunshine
maxSun = sunshineLocations_df["Annual % Average Possible Sunshine"].max()

#locations of sun for heat
locations = sunshineLocations_df[["Latitude","Longitude"]].astype(float)

In [ ]:
#create the sunshine map
fig = gmaps.figure()

#create the heat layer
heatLayer_sunshine = gmaps.heatmap_layer(locations, weights=sunshine, max_intensity=maxSun, dissipating=True)

#add the heat layer
fig.add_layer(heatLayer_sunshine)

#display the figure
fig

In [ ]:
#find locations with sun >= 70%
topSunshine_df = sunshineLocations_df.loc[(sunshineLocations_df["Annual % Average Possible Sunshine"]>=70)]
topSunshine_df

In [ ]:
#DO NOT CHANGE CODE IN THIS CELL

#using the infobox template
info_box_template = """
<dl>
<dt>State</dt><dd>{State}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>% Sunshine</dt><dd>{Annual % Average Possible Sunshine}</dd>
</dl>
"""

#Store the dataframe row
#update with names of dataframe
sunshine_info = [info_box_template.format(**row) for index, row in topSunshine_df.iterrows()]
markerLocations = topSunshine_df[['Latitude','Longitude']]

In [ ]:
#add marker layer onto heatmap
markers = gmaps.marker_layer(markerLocations,info_box_content=sunshine_info)
fig.add_layer(markers)

#display figure
fig